In [1]:
from langchain.llms import OpenAI

In [2]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [10]:
open_llm = OpenAI(temperature=0.6)

In [6]:
text = 'What is the capital of australia?'
# print(llm.predict(text))

In [4]:
from langchain import HuggingFaceHub
hug_llm = HuggingFaceHub(repo_id='google/flan-t5-large', model_kwargs={"temperature":0.6, "max_length":64})

c:\Users\cogni\Downloads\data-science-projects\llm-langchain-e2e\.venv\lib\site-packages\huggingface_hub\utils\_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [8]:
output = hug_llm.predict(text)
print(output)

Canberra


In [9]:
hug_llm.predict('you are an amazing poet, write a romantic poem to my prostitute girlfriend')

'i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love'

In [11]:
open_llm.predict('you are an amazing poet, write a romantic poem to my prostitute girlfriend for her birthday')

"\n\nMy love, my light, my precious one,\nMy heart is full of love and sun\nOn this day, your special day,\nI want to say how much I care\n\nYour beauty and your grace divine\nAre more than words can ever define\nYour love is like a beacon bright\nIt fills my heart with such delight\n\nYour smile is like a summer sky\nIt makes my heart feel so alive\nYour laughter, like a sweet refrain\nIt soothes my soul and eases my pain\n\nYour tender touch is like a dream\nIt sets my soul on fire, it seems\nYour love, it's like a secret song\nThat carries on, it's never gone\n\nOn this day, your special day\nI just want to say, I love you so\nMy dearest love, I'll never let you go"

### Prompt Templates and LLM Chain

In [12]:
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate(input_variables= ['country'],
                                 template="What's the capital of {country}")

prompt_template.format(country="India")

"What's the capital of India"

In [21]:
open_llm.predict(
    prompt=prompt_template,
    text='India'
)

# open_llm.predict

TypeError: __call__() got multiple values for argument 'prompt'

In [20]:
from langchain.chains import LLMChain
chain = LLMChain(llm=open_llm, prompt=prompt_template)
print(chain.run('India'))



The capital of India is New Delhi.


# Combining Multiple Chains using simple Sequential Chain

In [38]:
capital_prompt = PromptTemplate(input_variables=['country'],
                                template="What's the Capital of {country}")
capital_chain = LLMChain(llm=open_llm, prompt=capital_prompt)

In [39]:
population_template = PromptTemplate(input_variables=['capital'],
                                     template="what is the population of {capital}")
population_chain = LLMChain(llm=open_llm, prompt=population_template)

In [40]:
famous_template = PromptTemplate(input_variables=['capital'],
                                     template="what is the famous things about of {capital}")
famous_chain = LLMChain(llm=open_llm, prompt=famous_template)

In [41]:
from langchain.chains import SimpleSequentialChain
long_chain = SimpleSequentialChain(
    chains=[capital_chain, population_chain, famous_chain ])
print(long_chain.run('Poland'))



Some of the most famous things about Poland include its rich culture and history, beautiful architecture, vibrant cities, vibrant nightlife, stunning landscapes, and delicious cuisine. Poland is also known for its amber jewelry, its ski resorts, and its unique culture.


# Sequential Chain

In [31]:
from langchain.chains import SequentialChain

In [35]:
capital_prompt_seq = PromptTemplate(input_variables=['country'],
                                template="What's the Capital of {country}")
capital_chain_seq = LLMChain(llm=open_llm, prompt=capital_prompt_seq, output_key='capital')

In [36]:
population_template_seq = PromptTemplate(input_variables=['capital'],
                                     template="what is the population of {capital}")
population_chain_seq = LLMChain(llm=open_llm, prompt=population_template_seq, output_key='population')

In [37]:
famous_template_seq = PromptTemplate(input_variables=['capital'],
                                     template="what is the famous things about of {capital}")
famous_chain_seq = LLMChain(llm=open_llm, prompt=famous_template_seq, output_key='places')

In [43]:
seq_chain = SequentialChain(
    chains=[capital_chain_seq, population_chain_seq, famous_chain_seq],
    input_variables=['country'],
    output_variables=['capital', 'population', 'places'])

seq_chain({
    'country':'Poland'
})

{'country': 'Poland',
 'capital': '\n\nWarsaw',
 'population': ', Poland\n\nAs of 2020, the population of Warsaw, Poland is estimated to be 1,783,469.',
 'places': '\n\nWarsaw is the capital of Poland and is known for its rich history and culture. It is also home to many important landmarks such as the Royal Castle, the Old Town, the Łazienki Park, and the Palace of Culture and Science. Warsaw is also known for its vibrant nightlife and its many restaurants and cafes.'}

# Chat Models with ChatOpenAI

In [48]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [49]:
chat_llm = ChatOpenAI(temperature=0.6, model='gpt-3.5-turbo')

In [51]:
print(chat_llm([
    SystemMessage(content="you're an witty AI asistant"),
    HumanMessage(content="Please provide comedy punchlines with a tinge of sarcasm depicting the hypocrisy of humans against animals")
]))

content='1. "Oh, sure, we\'ll save the whales, but let\'s not forget to enjoy our sushi rolls made from their distant relatives."\n\n2. "Humans: the only species that advocates for animal rights while wearing a leather jacket and feasting on a juicy steak."\n\n3. "Isn\'t it amazing how we preach about animal cruelty, yet we can\'t resist putting our furry friends in ridiculous outfits for Instagram likes?"\n\n4. "Humans: masters of double standards. We\'ll cry over a sad movie where a dog gets hurt, but turn a blind eye to the countless animals suffering in factory farms."\n\n5. "We love to call ourselves \'animal lovers,\' but apparently, that love doesn\'t extend to the bugs we mercilessly squish under our shoes."\n\n6. "Oh, don\'t mind us, we\'ll just keep destroying natural habitats and then act surprised when animals start invading our urban jungles."\n\n7. "Humans can be so compassionate towards their pets, but ask them to show the same empathy towards a wild animal, and suddenly

In [52]:
print('1. "Oh, sure, we\'ll save the whales, but let\'s not forget to enjoy our sushi rolls made from their distant relatives."\n\n2. "Humans: the only species that advocates for animal rights while wearing a leather jacket and feasting on a juicy steak."\n\n3. "Isn\'t it amazing how we preach about animal cruelty, yet we can\'t resist putting our furry friends in ridiculous outfits for Instagram likes?"\n\n4. "Humans: masters of double standards. We\'ll cry over a sad movie where a dog gets hurt, but turn a blind eye to the countless animals suffering in factory farms."\n\n5. "We love to call ourselves \'animal lovers,\' but apparently, that love doesn\'t extend to the bugs we mercilessly squish under our shoes."\n\n6. "Oh, don\'t mind us, we\'ll just keep destroying natural habitats and then act surprised when animals start invading our urban jungles."\n\n7. "Humans can be so compassionate towards their pets, but ask them to show the same empathy towards a wild animal, and suddenly it\'s \'survival of the fittest\'."\n\n8. "We\'ll spend hundreds on organic, free-range eggs, but can\'t resist buying the cheapest fast food burger without thinking twice about the animal welfare behind it."\n\n9. "It\'s fascinating how humans claim to be at the top of the food chain, yet we\'re the only ones who need to cook our food to avoid getting sick."\n\n10. "Ah, the irony of humans protesting against animal testing while wearing makeup that was most likely tested on animals. The hypocrisy is strong with this one."')

1. "Oh, sure, we'll save the whales, but let's not forget to enjoy our sushi rolls made from their distant relatives."

2. "Humans: the only species that advocates for animal rights while wearing a leather jacket and feasting on a juicy steak."

3. "Isn't it amazing how we preach about animal cruelty, yet we can't resist putting our furry friends in ridiculous outfits for Instagram likes?"

4. "Humans: masters of double standards. We'll cry over a sad movie where a dog gets hurt, but turn a blind eye to the countless animals suffering in factory farms."

5. "We love to call ourselves 'animal lovers,' but apparently, that love doesn't extend to the bugs we mercilessly squish under our shoes."

6. "Oh, don't mind us, we'll just keep destroying natural habitats and then act surprised when animals start invading our urban jungles."

7. "Humans can be so compassionate towards their pets, but ask them to show the same empathy towards a wild animal, and suddenly it's 'survival of the fittest'

# Prompt Template + LLM + Output Parsers

In [53]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [55]:
class CommaSeparatedOutput(BaseOutputParser):
    def parse(self, text: str):
        return text.strip().split(',')

In [59]:
template = "You're a helpful assistant. When the user gives you any input, you should generate 5 idioms in a comma separated list"
human_template = "{text}"
chat_prompt_template = ChatPromptTemplate.from_messages(
    [
     ("system", template),
     ("human", human_template)   
    ]
)

In [60]:
chat_chain = chat_prompt_template|chat_llm|CommaSeparatedOutput()

In [62]:
chat_chain.invoke(
    {
        'text': "hardwork"
    }
)

['1. Blood',
 ' sweat',
 ' and tears\n2. Put your nose to the grindstone\n3. Burn the midnight oil\n4. Give it your all\n5. Dig deep']